1. Importar librerías

In [1]:
# PEP8 es una guía de estilo para escribir código Python. (https://www.python.org/dev/peps/pep-0008/)

# Librerías estándar de python
import os
import tarfile
import urllib.request as request
import pandas as pd
# Librerías de terceros
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Librerías locales
# from mi_clase import AwsStorage

%matplotlib inline

In [2]:
import sys
print(sys.executable)


d:\OneDrive - Landers\Documentos\Curso Python\parte_2\Sln_PublicarModelodeRegresion\venv\Scripts\python.exe


In [3]:
import os

# Carpeta donde están los datos
DATA_DIR = os.path.join("data")

# Ruta completa al archivo
INGRESOS_PATH = os.path.join(DATA_DIR, "ingresos.txt")


In [4]:
print(INGRESOS_PATH)

data\ingresos.txt


3. Extracción de los datos

In [5]:
import os

# Ruta a la carpeta de datos
DATA_DIR = os.path.join("data")
# Ruta completa al archivo
INGRESOS_PATH = os.path.join(DATA_DIR, "ingresos.txt")

def cargar_datos(path: str = INGRESOS_PATH) -> pd.DataFrame:
    # Verifica que el archivo exista
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Archivo no encontrado en: {path}")
    
    # Carga el archivo CSV y lo devuelve como DataFrame
    return pd.read_csv(path)

In [13]:
print(DATA_DIR)

data


In [15]:
print("Directorio actual:", os.getcwd())

Directorio actual: d:\OneDrive - Landers\Documentos\Curso Python\parte_2\Sln_PublicarModelodeRegresion\notebooks


In [16]:
os.chdir( 'd:\OneDrive - Landers\Documentos\Curso Python\parte_2\Sln_PublicarModelodeRegresion')

In [21]:
# Llamamos a la función para descargar los datos
data=cargar_datos()

In [22]:
print(data.head())

   edad nivel_educativo  horas_trabajadas  ingreso_mensual
0    56      secundaria              48.9          1673.99
1    46      secundaria              46.1          1614.62
2    32      secundaria              32.8          1398.33
3    60     universidad              37.3          2305.59
4    25     universidad              38.6          2037.76


4. Exploración de datos

4.1 Tipos de datos

In [23]:
# Resumen del dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   edad              100 non-null    int64  
 1   nivel_educativo   100 non-null    object 
 2   horas_trabajadas  100 non-null    float64
 3   ingreso_mensual   100 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 3.2+ KB


In [25]:
# Para el caso de la variable `nivel_educativo`, que es de tipo objeto (string), podemos utilizar el método `value_counts()`
# para ver la cantidad de ocurrencias de cada categoría.
data["nivel_educativo"].value_counts()

nivel_educativo
secundaria     43
universidad    41
posgrado       16
Name: count, dtype: int64

Se observa que las categorías 'secundaria' y 'universidad' están equilibradas porque tienen aproximadamente la misma cantidad de observaciones, la categoría 'posgrado' presenta un desequilibrio, dado que lo que se busca es predecir el 'ingreso' la cual es una variable numérica, usando ElasticNET (Regresión), no va a ser necerario balancearla.